In [1]:
import os
import torch
import numpy as np
import pandas as pd

from tqdm import tqdm

# Configuration

In [45]:
config = {
    'data_root': './data',
    'batch_size': 64,
    'num_worker': 0,
    
    'mlp_arch': [9, 16, 16, 8, 1],
    
    'trainer': 'Adam',
    'num_epoch': 5000,
    'learning_rate': 0.001,
    'early_stop_epoch': 10,
    'save_period': 1
}

# Utils

In [5]:
from torch.utils.data import random_split

In [6]:
def train_valid_split(full_dataset, valid_ratio):
    train_set, valid_set = random_split(full_dataset, [1-valid_ratio, valid_ratio])
    return train_set, valid_set

# Dataset

In [7]:
from torch.utils.data import Dataset

In [8]:
class TitanicDataset(Dataset):
    def __init__(self, data_root, training=False, transform=None):
        self.data_root = data_root
        self.training = training
        self.transform = transform
        
        if training:
            file_name = 'train.csv'
        else:
            file_name = 'test.csv'
            
        self.dataset = pd.read_csv(os.path.join(data_root, file_name))
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):  
        pclass = np.array(self.dataset['Pclass'][idx], dtype=np.float64)
        name = self.dataset['Name'][idx] # not used
        sex = np.array(pd.get_dummies(self.dataset['Sex']).iloc[idx], dtype=np.float64)
        age = np.array(self.dataset['Age'][idx] if not np.isnan(self.dataset['Age'][idx]) else -1, dtype=np.float64)
        sibsp = np.array(self.dataset['SibSp'][idx], dtype=np.float64)
        parch = np.array(self.dataset['Parch'][idx], dtype=np.float64)
        ticket = self.dataset['Ticket'][idx] # not used
        fare = np.array(self.dataset['Fare'][idx], dtype=np.float64)
        cabin = self.dataset['Cabin'][idx] # not used
        embarked = np.array(pd.get_dummies(self.dataset['Embarked']).iloc[idx], dtype=np.float64)
        
        if self.training:
            label = np.array(self.dataset['Survived'][idx], dtype=np.float64).item()
        else:
            label = None
        
        # feature: [paclass, female, male, age, sibsp, parch, fare, embarked(C), embarked(Q), embarked(S)]
        feature = np.concatenate((pclass, sex, age, sibsp, parch, fare, embarked), axis=None)
        sample = {'feature': feature, 'label': label}
        if self.transform:
            return self.transform(sample)
        else:
            return sample

In [9]:
dataset = TitanicDataset(data_root=config['data_root'], training=True)
train_set, valid_set = train_valid_split(dataset, 0.1)
dataset.__getitem__(3)

{'feature': array([ 1. ,  1. ,  0. , 35. ,  1. ,  0. , 53.1,  0. ,  0. ,  1. ]),
 'label': 1.0}

# Data Loader

In [10]:
from torch.utils.data import DataLoader

## Transforms

In [11]:
class ToTensor():

    def __init__(self) -> None:
        pass

    def __call__(self, data: dict) -> dict:
        for k, v in data.items():
            data[k] = torch.from_numpy(v).to(dtype=torch.get_default_dtype())
        return data

# Network

In [12]:
import torch.nn as nn

In [33]:
class SurvivalNet(nn.Module):
    
    def __init__(self, mlp_arch):
        super().__init__()
        self.mlp = nn.Sequential(*[
            self.mlp_block(mlp_arch[i-1], mlp_arch[i]) for i in range(1, len(mlp_arch)-1)
        ])
        self.output_layer = nn.Sequential(
            nn.Linear(mlp_arch[-2], mlp_arch[-1]),
            nn.Softmax()
        )
    
    def forward(self, x):
        x = self.mlp(x)
        x = self.output_layer(x)
        return x
    
    def mlp_block(self, dim_input, dim_output):
        return nn.Sequential(
            nn.Linear(dim_input, dim_output),
            nn.ReLU()
        )

In [39]:
net = SurvivalNet([2,2,1])
input_t = torch.Tensor([1.0, 2.0])
print(net(input_t))

tensor([1.], grad_fn=<SoftmaxBackward0>)


# Loss

In [14]:
loss = nn.CrossEntropyLoss()

# Metrics

In [15]:
def accurancy(prediction, target):
    assert len(prediction) == len(target)
    predition = torch.round(prediction)
    correct = torch.sum(torch.abs(prediction - target))
    return correct / len(target)

# Trainer

In [16]:
from torch.utils.tensorboard import SummaryWriter

In [52]:
class Trainer:
    
    def __init__(self,
        num_epoch, train_data_loader, valid_data_loader, model, optimizer, lr_scheduler, criterion, metric, early_stop_epoch, save_period, device
    ):
        self.num_epoch = num_epoch
        
        self.train_data_loader = train_data_loader
        self.valid_data_loader = valid_data_loader
        self.model = model
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.criterion = criterion
        self.metric = metric
        self.early_stop_epoch = early_stop_epoch
        self.save_period = save_period
        self.device = device
        
        self.acc_best = 0
        self.not_improved_count = 0
        self.writer = SummaryWriter('./run/log')
        self.early_stop = False
    
    def train(self):
        for epoch in range(self.num_epoch):
            self.__train_epoch(epoch)
            self.__valid_epoch(epoch)
    
    def __train_epoch(self, epoch):
        self.model.train()
        for batch_idx, data in tqdm(enumerate(self.train_data_loader), desc=f'Epoch {epoch}: '):
            feature = data['feature'].to(self.device)
            label = data['label'].to(self.device)
            print(type(feature))
            
            self.optimizer.zero_grad()
            output = self.model(feature)
            loss = self.criterion(output, label)
            loss.backward()
            self.optimizer.step()
            
            # record improved or not
            acc = self.metric(output, label)
            improved = (acc > self.acc_best)
            if improved:
                self.not_improved_count = 0
                self.acc_best = acc
            else:
                self.not_improved_count += 1
            
            # early stop
            if self.not_improved_count >= self.early_stop:
                self.early_stop = True
            
            # learning rate scheduler
            if lr_scheduler:
                self.lr_scheduler.step()
            
            # save model
            if epoch % self.save_period == 0:
                torch.save(model.state_dict(), './run/model')
            
            # tensorboard
            self.writer.add_scalar('Loss/Train loss', loss.item(), epoch)
            self.writer.add_scalar('Accuracy/Train acc', acc, epoch)
    
    def __valid_epoch(self, epoch):
        self.model.eval()
        with torch.no_grad():
            for batch_idx, data in enumerate(self.valid_data_loader):
                feature = data['feature'].to(self.device)
                label = data['label'].to(self.device)

                output = self.model(feature)
                loss = self.criterion(output, label)
                acc = self.metric(output, label)


                # tensorboard
                self.writer.add_scalar('Loss/Valid loss', loss.item(), epoch)
                self.writer.add_scalar('Accuracy/Valid acc', acc, epoch)

# Train

In [53]:
dataset = TitanicDataset(data_root=config['data_root'], training=True)
train_set, valid_set = train_valid_split(dataset, 0.1)

train_dataloader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True, num_workers=config['num_worker'])
valid_dataloader = DataLoader(valid_set, batch_size=config['batch_size'], shuffle=True, num_workers=config['num_worker'])

model = SurvivalNet(config['mlp_arch'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

loss = nn.CrossEntropyLoss()

trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(
    trainable_params,
    lr=config['learning_rate'],
    weight_decay=0,
    amsgrad=True
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=50,
    gamma=0.1
)

trainer = Trainer(
    num_epoch=config['num_epoch'],
    train_data_loader=train_dataloader,
    valid_data_loader=valid_dataloader,
    model=model,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    criterion=loss,
    metric=accurancy,
    early_stop_epoch=config['early_stop_epoch'],
    save_period = config['save_period'],
    device = device
)

trainer.train()

Epoch 0: : 0it [00:00, ?it/s]

<class 'torch.Tensor'>


RuntimeError: mat1 and mat2 must have the same dtype